In [ ]:
import requests
from pprint import pprint

# API URL
url = "https://comic.naver.com/api/webtoon/titlelist/weekday?order=user"

# HTTP GET 요청
response = requests.get(url)
response.raise_for_status()  # 요청 실패 시 예외 발생

# JSON 데이터 파싱
data = response.json()

# "titleListMap"에서 각 요일의 "titleId" 추출
title_ids = []
for day, webtoons in data["titleListMap"].items():
    for webtoon in webtoons:
        title_ids.append(webtoon["titleId"])

# 결과 출력
print(title_ids)


In [ ]:
import requests
from pprint import pprint

# API URL
url = "https://comic.naver.com/api/webtoon/titlelist/weekday?week=dailyPlus&order=user"

# HTTP GET 요청
response = requests.get(url)
response.raise_for_status()  # 요청 실패 시 예외 발생

# JSON 데이터 파싱
data = response.json()

# "titleList"에서 "titleId" 추출
dplus_ids = []


for webtoon in data["titleList"]:
    dplus_ids.append(webtoon["titleId"])

# 결과 출력
print(dplus_ids)

In [ ]:
import requests

fin_ids = []
i = 1
while i < 70:
    # API URL
    url = f"https://comic.naver.com/api/webtoon/titlelist/finished?page={i}&order=UPDATE"

    # HTTP GET 요청
    response = requests.get(url)
    response.raise_for_status()  # 요청 실패 시 예외 발생

    # JSON 데이터 파싱
    data = response.json()

    # 필요한 정보 추출 및 fin_ids에 추가
    for webtoon in data.get("titleList", []):
        title_id = webtoon.get("titleId")
        if title_id:  # titleId가 존재하면 추가
            fin_ids.append(title_id)

    i = i + 1

# 결과 출력
print(fin_ids)

In [ ]:
total_ids = fin_ids + title_ids + dplus_ids
print(len(title_ids))
print(len(fin_ids))
print(len(dplus_ids))
print(len(total_ids))

In [ ]:
ids_list = list(dict.fromkeys(total_ids)) #중복되는 id제거
len(ids_list)

In [103]:
# imports
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver


driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)


In [104]:
driver.get("https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/") 
# naver 아이디 비밀번호로 로그인할것.


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import json
from selenium.webdriver.common.by import By

all_data = []
for i in ids_list:
    # API URL
    URL = f"https://comic.naver.com/api/article/list/info?titleId={i}"

    # HTTP GET 요청
    response = requests.get(URL)
    response.raise_for_status()  # 요청 실패 시 예외 발생

    # JSON 데이터 파싱
    data = response.json()

    artist_info = {}
    for artist in data.get("communityArtists", []):
        artist_type_list = artist.get("artistTypeList", [])
        artist_name = artist.get("name")
        for artist_type in artist_type_list:  # artistTypeList에 여러 값이 있을 경우 처리
            if artist_type not in artist_info:
                artist_info[artist_type] = []
            artist_info[artist_type].append(artist_name)
    publishDescription = data.get("publishDescription")
    if data.get("rest") == True:
        publishDescription = "휴재"
        
    drawing = "-"
    author = "-"
    original = "-"
    price = "무료"
    for artist in data.get("communityArtists", []):
        artist_type_list = artist.get("artistTypeList", [])
        artist_name = artist.get("name")
        for artist_type in artist_type_list:
            if artist_type == "ARTIST_PAINTER":
                drawing = artist_name
            elif artist_type == "ARTIST_WRITER":
                author = artist_name
            elif artist_type == "ARTIST_NOVEL_ORIGIN":
                original = artist_name

    
    driver.get(f"https://comic.naver.com/api/article/list?titleId={i}") # 소스 URL
    d_data = driver.page_source

    soup = BeautifulSoup(d_data, "html.parser")
    # soup 객체는 이미 생성되어 있다고 가정
    pre_tags = soup.select("pre")

    # pre 태그 내 첫 번째 데이터 가져오기
    if pre_tags:
        json_data = pre_tags[0].get_text()  # pre 태그의 텍스트를 가져옴
        
        # JSON 형식으로 변환
        data_dict = json.loads(json_data)  # JSON 문자열을 파이썬 딕셔너리로 변환
        
    # 브라우저 열기
    # driver = webdriver.Chrome(driver_path)
    driver.get(f"https://comic.naver.com/webtoon/list?titleId={i}")

    # li[20] 대신 모든 li를 순회하며 확인
    elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[3]/ul/li/a/div[1]/i/span')

    if len(elements) > 0:
        price = "유료"
    if data.get("dailyPass") == True:
        price = "24시간 기다리면 무료"

    
    
    # 필요한 정보만 추출
    selected_data = {
        "id": i,
        "type": "웹툰",
        "platform" : "네이버 웹툰",
        "title": data.get("titleName"),
        "status": publishDescription,
        "thumbnail": data.get("thumbnailUrl"),
        "genre": data.get("curationTagList", [{}])[0].get("tagName", "") if data.get("curationTagList") else "",
        "views":"-",
        "description": data.get("synopsis"),
        "keywords": data.get("gfpAdCustomParam", {}).get("tags", []),
        "author": author,     # ARTIST_WRITER 아티스트 이름
        "illustrator": drawing,   # ARTIST_PAINTER 아티스트 이름
        "original": original,
        "age_rating": data.get("age", {}).get("description", []),
        "price": price,
        "favoriteCount": data.get("favoriteCount"),
        "url":f"https://comic.naver.com/webtoon/list?titleId={i}",
        "episode" : data_dict.get("totalCount")
    }


    # selected_data를 all_data에 추가
    all_data.append(selected_data)

# JSON 형식으로 파일 저장
# with open('selected_data.json', 'w', encoding='utf-8') as f:
#     json.dump(all_data, f, ensure_ascii=False, indent=4)

    # # 데이터 출력
    # print(f"제목: {selected_data['titleName']}")
    # print(f"이미지URL: {selected_data['thumbnailUrl']}")
    # print(f"요일분류: {selected_data['publishDescription']}")
    # print(f"관심: {selected_data["favoriteCount"]}")
    # print(f"줄거리: {selected_data['synopsis']}\n")

    # print("작가:")
    # for artist in selected_data["communityArtists"]:
    #     print(f"- Name: {artist.get('name')}, Type: {', '.join(artist.get('artistTypeList', []))}")

    # print("\n태그:")
    # for tag in selected_data["tags"]:
    #     print(f"- {tag}")



In [ ]:
len(all_data)

In [ ]:
#별점 구하기
import requests

rating_data = []
genres = ["PURE", "FANTASY", "ACTION", "DAILY", "THRILL", "COMIC", "HISTORICAL", "DRAMA","SENSIBILITY","SPORTS"]
page_nums = [str(j) for j in range(1, 61)]

for genre in genres:
    for page_num in page_nums:
        url = f"https://comic.naver.com/api/webtoon/titlelist/genre?type=GENRE&genre={genre}&page={page_num}&pageSize=25&order=USER"
        
        try:
            # HTTP GET 요청
            response = requests.get(url)
            response.raise_for_status()  # HTTP 에러 발생 시 예외
            
            # JSON 데이터 파싱
            data = response.json()
            
            # "titleList"에 있는 각 웹툰의 정보를 추출
            for webtoon in data.get("titleList", []):
                # ids = webtoon.get("titleId")
                starscore = webtoon.get("starScore")
                thumbnail = webtoon.get("thumbnailUrl")
                
                # 필요한 데이터를 딕셔너리로 저장
                selected_data = {
                    # "id":ids,
                    "thumbnail": thumbnail,
                    "rating": starscore
                }
                
                rating_data.append(selected_data)
        
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for genre={genre}, page={page_num}: {e}")
        except KeyError as e:
            print(f"Missing expected data in response for genre={genre}, page={page_num}: {e}")

# 결과 출력
print(rating_data)

In [115]:
rating_data = [dict(t) for t in {frozenset(d.items()) for d in rating_data}]

In [ ]:
len(rating_data)

In [ ]:
def merge_by_thumbnail(rating_data, all_data):
    merged_dict = {}

    # 1) A_list를 순회하며 thumbnail을 key로 하는 merged_dict 생성
    for A in rating_data:
        thumb = A.get('thumbnail')
        if thumb:  # thumbnail이 존재하는지 확인
            merged_dict[thumb] = A
        else:
            # thumbnail key가 없을 수 있으므로 예외 처리 또는 직접 append
            pass

    # 2) B_list를 순회하며 thumbnail이 겹치는지 확인 후 병합
    for B in all_data:
        thumb = B.get('thumbnail')
        if thumb in merged_dict:
            # 같은 thumbnail이 이미 있으면 합쳐서 업데이트
            merged_dict[thumb] = {**merged_dict[thumb], **B}
        else:
            # 새로운 thumbnail이면 그대로 추가
            merged_dict[thumb] = B

    # 3) 딕셔너리의 value 부분만 꺼내 리스트로 변환
    return list(merged_dict.values())


# 위 예시 데이터를 병합해보기
merged_list = merge_by_thumbnail(rating_data, all_data)


for i, item in enumerate(merged_list, start=1):
    print(f"병합 결과 {i}: {item}")

In [ ]:
len(merged_list)

In [119]:
# JSON 형식으로 파일 저장
with open('navertoon_data.json', 'w', encoding='utf-8') as f:
    json.dump(merged_list, f, ensure_ascii=False, indent=4)